In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Embedding, LSTM, Flatten, Concatenate, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
# Load your dataset
df = pd.read_csv('generated_dataset.csv')

In [8]:
# Encode categorical variables
df['UserGender'] = df['UserGender'].map({'M': 0, 'F': 1})

In [11]:
# Tokenize and pad liked topics
max_topics_length = 50
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['LikedTopic'])  
topics_seq = tokenizer.texts_to_sequences(df['LikedTopic'])  
topics_pad = pad_sequences(topics_seq, maxlen=max_topics_length)

In [21]:
# Split the data
X_age = df['UserAge'].values
X_gender = df['UserGender'].values
X_topics = topics_pad
y = np.ones(len(df))  # Binary label: 1 for 'liked', assuming all entries are liked

In [35]:
# Split the data into train and test sets
X_age_train, X_age_test, X_gender_train, X_gender_test, X_topics_train, X_topics_test, y_train, y_test = train_test_split(
    X_age, X_gender, X_topics, y, test_size=0.2, random_state=42)

In [38]:
# Define the model
input_age = Input(shape=(1,), name='age_input')
input_gender = Input(shape=(1,), name='gender_input')
input_topics = Input(shape=(max_topics_length,), name='topics_input')

In [39]:
embedding_age = Embedding(input_dim=100, output_dim=16)(input_age)  # Adjust input_dim as needed
embedding_gender = Embedding(input_dim=2, output_dim=16)(input_gender)
embedding_topics = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=16)(input_topics)

In [40]:
lstm_age = LSTM(16)(embedding_age)
lstm_gender = LSTM(16)(embedding_gender)
lstm_topics = LSTM(16)(embedding_topics)

In [41]:
concatenated = Concatenate()([lstm_age, lstm_gender, lstm_topics])
output = Dense(1, activation='sigmoid')(concatenated)

In [42]:
model = Model(inputs=[input_age, input_gender, input_topics], outputs=output)

In [43]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [44]:
# Train the model
model.fit(
    {'age_input': X_age_train, 'gender_input': X_gender_train, 'topics_input': X_topics_train},
    y_train,
    epochs=10,
    batch_size=64,
    validation_split=0.2
)

Epoch 1/10
6/6 [==============================] - 9s 324ms/step - loss: 0.6843 - accuracy: 0.8128 - val_loss: 0.6658 - val_accuracy: 1.0000
Epoch 2/10
6/6 [==============================] - 0s 25ms/step - loss: 0.6548 - accuracy: 1.0000 - val_loss: 0.6354 - val_accuracy: 1.0000
Epoch 3/10
6/6 [==============================] - 0s 22ms/step - loss: 0.6227 - accuracy: 1.0000 - val_loss: 0.5990 - val_accuracy: 1.0000
Epoch 4/10
6/6 [==============================] - 0s 25ms/step - loss: 0.5820 - accuracy: 1.0000 - val_loss: 0.5490 - val_accuracy: 1.0000
Epoch 5/10
6/6 [==============================] - 0s 22ms/step - loss: 0.5227 - accuracy: 1.0000 - val_loss: 0.4701 - val_accuracy: 1.0000
Epoch 6/10
6/6 [==============================] - 0s 23ms/step - loss: 0.4262 - accuracy: 1.0000 - val_loss: 0.3461 - val_accuracy: 1.0000
Epoch 7/10
6/6 [==============================] - 0s 22ms/step - loss: 0.2972 - accuracy: 1.0000 - val_loss: 0.2266 - val_accuracy: 1.0000
Epoch 8/10
6/6 [==========